In [ ]:
# Extract bits n-m of a virtual address x
def extract(x, n, m):
    return (x >> m) & (2**(n-m+1)-1)

# Three CRC hash functions H1, H2, and H3 that operate on pte_tag
def h1(x, len):
    return crc(x) % len

def h2(x, len):
    return crc(x ^ 0xFFFFFFFF)  % len

def h3(x, len):
    return crc(x ^ 0xAAAAAAAA)  % len

# CRC hash function for 33-bit integers
def crc(x):
    crc = 0xFFFFFFFF
    for i in range(33):
        if (((x >> i) & 1) ^ (crc >> 31)) == 1:
            crc = ((crc << 1) ^ 0x04C11DB7) & 0xFFFFFFFF
        else:
            crc = (crc << 1) & 0xFFFFFFFF
    return crc

# Takes the value performs the hashing and stores in a list 
def ilist_gen(x, len):
    i1 = h1(x, len)
    i2 = h2(x, len)  
    i3 = h3(x, len)
    return [i1, i2, i3]

In [ ]:
import random
import numpy as np

# d-ary hash table with len entries in each ary 
len = 8192                     # Length of each hash table
bits = 3                       # Number of bits for offset 
entry_len = 2 ** bits + 1      # Length of each entry in the hash table
ary = 3                        # Number of hash tables
max_attempts = 10              # Maximum number of attempts to insert a pte_tag
off_len = 3                    # Number of bits for offset

# Numpy array of -1s to represent the pte_table
pte_table = np.full((ary, len, entry_len), -1)

# Count of the number of page hits
count = [0]

# Function to insert a pte_tag into the pte_table
def insert(x):

    dlist = [i for i in range(ary)]

    prev_d = -1
    pte_tag = extract(x, 47, 12 + off_len)
    pte_offset = extract(x, 12 + off_len - 1, 12)

    for j in range(1, max_attempts):
        ilist = [h1(pte_tag, len), h2(pte_tag, len), h3(pte_tag, len)]
        for d in dlist: 
            if pte_table[d][ilist[d]][entry_len - 1] == pte_tag:
                if pte_table[d][ilist[d]][pte_offset] == -1: 
                    pte_table[d][ilist[d]][pte_offset] = 1
                    return j, True 
                else:
                    if j == 1: count[0] += 1
                    return j, True
            elif pte_table[d][ilist[d]][entry_len - 1] == -1:
                pte_table[d][ilist[d]][entry_len - 1] = pte_tag
                pte_table[d][ilist[d]][pte_offset] = 1
                return j, True
        
    
        curr_d = random.choice(dlist)
  
        if(prev_d != -1): dlist.append(prev_d)

        update_pte_tag = pte_tag
        
        pte_tag = pte_table[curr_d][ilist[curr_d]][entry_len - 1]

        pte_table[curr_d][ilist[curr_d]][entry_len - 1] = update_pte_tag
        pte_table[curr_d][ilist[curr_d]][pte_offset] = 1

        prev_d = curr_d
        dlist.remove(curr_d)
        
    return j, False

In [ ]:
import time
 
# Number of insertion failures and successes
yep = 0
nah = 0

prev_j = 1

start = time.time()
with open("pinatrace_gups.out") as file:
    i = 0
    for line in file:
        i += 1
        if line != '#eof\n':
            virtual_address = line[18:32]
            virtual_address = int(virtual_address, 0)
            j, outcome = insert(virtual_address)
                
            prev_j = (prev_j + j)/2

            if(outcome == False):
                nah += 1
            else:
                yep += 1

        if(i % 100000000 == 0):
            print(i, time.time() - start)
        
end =  time.time()

print("Execution time in seconds: ",(end-start))

In [ ]:
# Average attempts for insertion 
prev_j

In [ ]:
entry_null = 0 
entry_fill = 0 

total_null = 0 
total_fill = 0

for d in range(ary) :
    for i in range(len):
        if pte_table[d][i][8] == -1:
            entry_null += 1
        else: 
            entry_fill += 1
        for j in range(entry_len - 1):
            if pte_table[d][i][j] == -1:
                total_null += 1
            else:
                total_fill += 1

In [ ]:
# Number of filled entries in the hash table
entry_fill

In [ ]:
# Number of empty entries in the hash table
entry_null

In [ ]:
# Number of PT filled entries in the hash table
total_null

In [ ]:
# Number of PT empty entries in the hash table
total_fill

In [ ]:
# Insertion success rate
yep 

In [ ]:

# Insertion failure rate 
nah